# Mock Quiz Week6

In [23]:
# import sys
# import os
# import django

# sys.path.append('..')

# os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'university.settings')
# os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# django.setup()

# print("Django environment loaded successfully!")

import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from school.models import *
from datetime import date, datetime
from django.db.models import Q, F

### Setup
1. สร้าง virtual environment
2. ติดตั้ง django และ psycopg2 libraries
3. สร้างโปรเจคใหม่ใหม่ชื่อ `university`
4. จากนั้นให้ทำการ startapp ใหม่ชื่อ `school`
5. สร้าง database ชื่อ `school` ใน Postgres DB
6. ทำการเพิ่ม code ด้านล่างนี้ในไฟล์ `school/models.py`
7. เพิ่ม 'school' ใน settings.py
8. จากนั้นทำการติดตั้งเพื่อใช้งาน jupyter notebook ตามขั้นตอนใน django_notebook.md
9. ทำการ makemigrations และ migrate
10. import ข้อมูลจาก school.sql ลงไปใน database

```python
# school/models.py
from django.db import models


class Teacher(models.Model):
    first_name = models.CharField(max_length=100)
    last_name = models.CharField(max_length=100)
    email = models.EmailField(unique=True)

    def __str__(self):
        return f"{self.first_name} {self.last_name}"


class CourseCategory(models.Model):
    name = models.CharField(max_length=100, unique=True)
    description = models.TextField(blank=True, null=True)

    def __str__(self):
        return self.name


class Course(models.Model):
    title = models.CharField(max_length=200)
    description = models.TextField(blank=True, null=True)
    credits = models.PositiveIntegerField(default=3)
    teacher = models.ForeignKey(Teacher, on_delete=models.SET_NULL, null=True)
    categories = models.ManyToManyField(CourseCategory, blank=True)

    def __str__(self):
        return self.title


class Student(models.Model):
    first_name = models.CharField(max_length=100)
    last_name = models.CharField(max_length=100)
    student_id = models.CharField(max_length=10, unique=True)
    enrollment_date = models.DateField()
    courses = models.ManyToManyField(Course, blank=True)

    def __str__(self):
        return f"{self.first_name} {self.last_name}"


class StudentProfile(models.Model):
    student = models.OneToOneField(Student, on_delete=models.CASCADE, primary_key=True)
    bio = models.TextField(blank=True, null=True)
    profile_picture_url = models.URLField(blank=True, null=True)

    def __str__(self):
        return f"Profile of {self.student.first_name}"
```
***

### Part 1: Query

1. ให้ Query แสดงประเภทของคอร์ส (`CourseCategory`) ทั้งหมด

```
ID: 1, Name: Science, Description: Courses related to natural and physical sciences.
ID: 2, Name: Mathematics, Description: Courses focused on numbers, quantity, and space.
ID: 3, Name: Humanities, Description: Courses studying human culture, including literature, history, and philosophy.
ID: 4, Name: Technology, Description: Courses involving the application of scientific knowledge for practical purposes.
ID: 5, Name: Arts, Description: Courses related to creative expression and aesthetics.
ID: 6, Name: Business, Description: Courses focused on commerce, finance, and management.
ID: 7, Name: Health, Description: Courses related to medicine, wellness, and public health.
```

In [15]:
# code
allcat = CourseCategory.objects.all()
for i in allcat:
    print(f"ID: {i.id}, Name: {i.name}, Description: {i.description} ")

ID: 1, Name: Science, Description: Courses related to natural and physical sciences. 
ID: 2, Name: Mathematics, Description: Courses focused on numbers, quantity, and space. 
ID: 3, Name: Humanities, Description: Courses studying human culture, including literature, history, and philosophy. 
ID: 4, Name: Technology, Description: Courses involving the application of scientific knowledge for practical purposes. 
ID: 5, Name: Arts, Description: Courses related to creative expression and aesthetics. 
ID: 6, Name: Business, Description: Courses focused on commerce, finance, and management. 
ID: 7, Name: Health, Description: Courses related to medicine, wellness, and public health. 


2. ให้แสดงข้อมูลของคอร์สที่ชื่อ "Calculus I" (ห้ามดึงโดยการใช้ ID)
```
ID: 2, Title: Calculus I, Description: An introduction to differential calculus., Credits: 4
```

In [22]:
# code
course = Course.objects.get(title = "Calculus I")
print(f"ID: {course.id}, TItle: {course.title}, Description: {course.description}, Credits: {course.credits}")

ID: 2, TItle: Calculus I, Description: An introduction to differential calculus., Credits: 4


3. ให้แสดงรายชื่อนักเรียนที่มีวันลงทะเบียน (`enrollment_date`) เป็นวันที่ 1 กันยายน 2023 (ต้องมีการใช้ date object)
```
ID: 1, Name: Alice Smith, Enrollment Date: 2023-09-01
ID: 2, Name: Bob Johnson, Enrollment Date: 2023-09-01
ID: 14, Name: Nora Wilson, Enrollment Date: 2023-09-01
```

In [26]:
# code
thisdate = date(2023,9,1)
allStudent = Student.objects.filter(enrollment_date = thisdate)

for stu in allStudent:
    print(f"ID: {stu.id}, Name: {stu.first_name} {stu.last_name}, Enrollment Date: {stu.enrollment_date}")

ID: 1, Name: Alice Smith, Enrollment Date: 2023-09-01
ID: 2, Name: Bob Johnson, Enrollment Date: 2023-09-01
ID: 14, Name: Nora Wilson, Enrollment Date: 2023-09-01


4. แสดงชื่ออาจารย์ที่สอนวิชา "Introduction to Physics" (Query ได้แค่ครั้งเดียว)
```
ID: 1, Title: Introduction to Physics, Teacher: Michael Johnson
```

In [51]:
# code
course = Course.objects.get(title = "Introduction to Physics")
print(f"ID: {course.id}, Title: {course.title}, Teacher: {course.teacher.first_name} {course.teacher.last_name}")


ID: 1, Title: Introduction to Physics, Teacher: Michael Johnson


6. แสดงชื่อวิชาทั้งหมดที่อาจารย์ "Emily Williams" เป็นคนสอน
```
ID: 2, Title: Calculus I
ID: 8, Title: Linear Algebra
```

In [54]:
# code
allCourse = Course.objects.filter(teacher__first_name = "Emily", teacher__last_name = "Williams")
for i in allCourse:
    print(f"ID: {i.id}, Title: {i.title}")

ID: 2, Title: Calculus I
ID: 8, Title: Linear Algebra


7. แสดงชื่อวิชาทั้งหมดที่อยู่ในหมวดหมู่ "Technology" (ต้องมีการใช้ _set)
```
ID: 1, Title: Introduction to Physics
ID: 4, Title: Python Programming Fundamentals
ID: 10, Title: Web Development with Django
ID: 11, Title: Digital Photography
ID: 14, Title: Data Structures and Algorithms
```

In [59]:
# code
cat = CourseCategory.objects.get(name = "Technology")
allCourse = cat.course_set.all()
for i in allCourse:
    print(f"ID: {i.id}, Title: {i.title}")

ID: 1, Title: Introduction to Physics
ID: 4, Title: Python Programming Fundamentals
ID: 10, Title: Web Development with Django
ID: 11, Title: Digital Photography
ID: 14, Title: Data Structures and Algorithms


8. แสดงชื่อคอร์สทั้งหมดที่ชื่อคอร์สมีคำว่า "to" (To to tO นับหมด) และเรียง ID จากน้อยไปมาก
```
ID: 1, Title: Introduction to Physics
ID: 3, Title: World History: Ancient Civilizations
ID: 5, Title: Introduction To Drawing
ID: 11, Title: Digital Photography
ID: 12, Title: Introduction tO Marketing
ID: 13, Title: Human Anatomy
ID: 15, Title: Introduction to Philosophy
```

In [62]:
# code
allCourse = Course.objects.filter(title__icontains = "to")
for i in allCourse:
    print(f"ID: {i.id}, TItle: {i.title}")

ID: 1, TItle: Introduction to Physics
ID: 3, TItle: World History: Ancient Civilizations
ID: 5, TItle: Introduction To Drawing
ID: 11, TItle: Digital Photography
ID: 12, TItle: Introduction tO Marketing
ID: 13, TItle: Human Anatomy
ID: 15, TItle: Introduction to Philosophy


9. แสดงชื่อนักเรียนทั้งหมดที่ลงทะเบียนในวันที่ 5 และเรียงชื่อจาก z-a
```
ID: 8, Name: Hannah Davis, Enrollment Date: 2022-09-05
ID: 7, Name: George Miller, Enrollment Date: 2023-09-05
```

In [66]:
# code
allStudent = Student.objects.filter(enrollment_date__day = 5).order_by("-first_name")
for i in allStudent:
    print(f"ID: {i.id}, Name: {i.first_name} {i.last_name}, Enrollment Date: {i.enrollment_date}")

ID: 8, Name: Hannah Davis, Enrollment Date: 2022-09-05
ID: 7, Name: George Miller, Enrollment Date: 2023-09-05


10. แสดงชื่อวิชาทั้งหมดที่มีหน่วยกิตมากกว่า 3
```
ID: 1, Title: Introduction to Physics, Credits: 4
ID: 2, Title: Calculus I, Credits: 4
ID: 7, Title: Advanced Chemistry, Credits: 4
ID: 8, Title: Linear Algebra, Credits: 4
ID: 10, Title: Web Development with Django, Credits: 4
ID: 13, Title: Human Anatomy, Credits: 4
ID: 14, Title: Data Structures and Algorithms, Credits: 4
```

In [70]:
# code
allCourse = Course.objects.filter(credits__gt = 3)

for i in allCourse:
    print(f"ID: {i.id}, Title: {i.title}, Credits: {i.credits}")

ID: 1, Title: Introduction to Physics, Credits: 4
ID: 2, Title: Calculus I, Credits: 4
ID: 7, Title: Advanced Chemistry, Credits: 4
ID: 8, Title: Linear Algebra, Credits: 4
ID: 10, Title: Web Development with Django, Credits: 4
ID: 13, Title: Human Anatomy, Credits: 4
ID: 14, Title: Data Structures and Algorithms, Credits: 4


11. แสดงชื่อวิชาทั้งหมดที่มีหน่วยกิตอยู่ระหว่าง 2-3 รวมถึงแสดงชื่ออาจารย์แต่ละวิชา
```
ID: 3, Title: World History: Ancient Civilizations, Credits: 3, Teacher: Sarah Jones
ID: 4, Title: Python Programming Fundamentals, Credits: 3, Teacher: Chris Garcia
ID: 6, Title: Principles of Macroeconomics, Credits: 3, Teacher: Daniel Davis
ID: 9, Title: Literary Analysis, Credits: 3, Teacher: Sarah Jones
ID: 11, Title: Digital Photography, Credits: 3, Teacher: Jessica Miller
ID: 15, Title: Introduction to Philosophy, Credits: 3, Teacher: Robert Martinez
ID: 12, Title: Introduction tO Marketing, Credits: 3, Teacher: Daniel Davis
ID: 5, Title: Introduction To Drawing, Credits: 2, Teacher: Jessica Miller
```

In [78]:
# code
allCourse = Course.objects.filter(credits__range = (2,3))
for i in allCourse:
    print(f"ID: {i.id}, Title: {i.title}, Credits: {i.credits}, Teacher: {i.teacher.first_name} {i.teacher.last_name}")

ID: 3, Title: World History: Ancient Civilizations, Credits: 3, Teacher: Sarah Jones
ID: 4, Title: Python Programming Fundamentals, Credits: 3, Teacher: Chris Garcia
ID: 5, Title: Introduction To Drawing, Credits: 2, Teacher: Jessica Miller
ID: 6, Title: Principles of Macroeconomics, Credits: 3, Teacher: Daniel Davis
ID: 9, Title: Literary Analysis, Credits: 3, Teacher: Sarah Jones
ID: 11, Title: Digital Photography, Credits: 3, Teacher: Jessica Miller
ID: 12, Title: Introduction tO Marketing, Credits: 3, Teacher: Daniel Davis
ID: 15, Title: Introduction to Philosophy, Credits: 3, Teacher: Robert Martinez


12. แสดงชื่อนักเรียนที่ลงทะเบียนระหว่างเดือนมกราคม ถึง เดือนมีนาคม และเรียงลำดับปีจากน้อยไปมาก
```
ID: 11, Name: Kevin Hernandez, Enrollment Date: 2022-01-18
ID: 10, Name: Jane Martinez, Enrollment Date: 2023-01-20
ID: 6, Name: Fiona Garcia, Enrollment Date: 2024-01-15
ID: 9, Name: Ian Rodriguez, Enrollment Date: 2024-02-01
ID: 13, Name: Mason Gonzalez, Enrollment Date: 2024-03-10
```

In [83]:
# code
allStudent = Student.objects.filter(enrollment_date__month__gte = 1, enrollment_date__month__lte = 3).order_by("enrollment_date__year")

for i in allStudent:
    print(f"ID: {i.id}, Name: {i.first_name} {i.last_name}, Enrollment_date: {i.enrollment_date}")

ID: 11, Name: Kevin Hernandez, Enrollment_date: 2022-01-18
ID: 10, Name: Jane Martinez, Enrollment_date: 2023-01-20
ID: 6, Name: Fiona Garcia, Enrollment_date: 2024-01-15
ID: 9, Name: Ian Rodriguez, Enrollment_date: 2024-02-01
ID: 13, Name: Mason Gonzalez, Enrollment_date: 2024-03-10


13. แสดงชื่อนักเรียนที่ลงทะเบียนวิชา "Introduction to Physics" และมีวันลงทะเบียนในเดือนกันยายน
```
ID: 1, Name: Alice Smith, Enrollment Date: 2023-09-01
ID: 7, Name: George Miller, Enrollment Date: 2023-09-05
```

In [92]:
# code
physics = Course.objects.get(title = "Introduction to Physics")

for i in physics.student_set.all():
    print(f"ID: {i.id}, Name: {i.first_name} {i.last_name}, Enrollment Date: {i.enrollment_date}")

ID: 1, Name: Alice Smith, Enrollment Date: 2023-09-01
ID: 7, Name: George Miller, Enrollment Date: 2023-09-05


14. ค้นหาชื่อคอร์สที่ขึ้นต้นด้วย "Introduction"
```
ID: 1, Title: Introduction to Physics
ID: 15, Title: Introduction to Philosophy
ID: 12, Title: Introduction tO Marketing
ID: 5, Title: Introduction To Drawing
```

In [99]:
# code
allCourse = Course.objects.filter(title__startswith = "Introduction")
for i in allCourse:
    print(f"ID: {i.id}, Title: {i.title}")

ID: 1, Title: Introduction to Physics
ID: 5, Title: Introduction To Drawing
ID: 12, Title: Introduction tO Marketing
ID: 15, Title: Introduction to Philosophy


15. ค้นหานักเรียนที่ชื่อขึ้นต้นด้วยตัวอักษร "A" หรือ "D"
```
ID: 1, Name: Alice Smith
ID: 4, Name: Diana Brown
```

In [102]:
# code
allStudent = Student.objects.filter(Q(first_name__startswith = "A") | Q(first_name__startswith = "D"))
for i in allStudent:
    print(f"ID: {i.id}, Name: {i.first_name} {i.last_name}")

ID: 1, Name: Alice Smith
ID: 4, Name: Diana Brown


16. ค้นหาวิชาที่มีอาจารย์ "Michael" หรือ "Daniel" เป็นคนสอน (query ได้เพียงครั้งเดียว)
```
ID: 1, Title: Introduction to Physics, Teacher: Michael Johnson
ID: 6, Title: Principles of Macroeconomics, Teacher: Daniel Davis
ID: 7, Title: Advanced Chemistry, Teacher: Michael Johnson
ID: 12, Title: Introduction tO Marketing, Teacher: Daniel Davis
```

In [107]:
# code
allCourse = Course.objects.filter(Q(teacher__first_name = "Michael") | Q(teacher__first_name = "Daniel"))
for i in allCourse:
    print(f"ID: {i.id}, Title: {i.title}, Teacher: {i.teacher.first_name} {i.teacher.last_name}")

ID: 1, Title: Introduction to Physics, Teacher: Michael Johnson
ID: 6, Title: Principles of Macroeconomics, Teacher: Daniel Davis
ID: 7, Title: Advanced Chemistry, Teacher: Michael Johnson
ID: 12, Title: Introduction tO Marketing, Teacher: Daniel Davis


17. ค้นหาวิชาที่มีอาจารย์ "Michael" เป็นคนสอน และวิชานั้น ๆ ขึ้นต้นด้วย "Introduction" หรือลงท้ายด้วย "Chemistry" (query ได้เพียงครั้งเดียว)
```
ID: 1, Title: Introduction to Physics, Teacher: Michael Johnson
ID: 7, Title: Advanced Chemistry, Teacher: Michael Johnson
```

In [118]:
# code
allCourse = Course.objects.filter(Q(title__startswith = "Introduction") | Q(title__endswith = "Chemistry"), teacher__first_name = "Michael")
for i in allCourse:
    print(f"ID: {i.id}, Title: {i.title}, Teacher: {i.teacher.first_name} {i.teacher.last_name}")

ID: 1, Title: Introduction to Physics, Teacher: Michael Johnson
ID: 7, Title: Advanced Chemistry, Teacher: Michael Johnson


18. ค้นหานักเรียนที่มีวันลงทะเบียนระหว่างวันที่ 2023-09-02 ถึง 2024-02-01 และแสดงผลโดยเรียงวันจากน้อยไปมาก
```
ID: 12, Name: Laura Lopez, Enrollment Date: 2023-09-02
ID: 7, Name: George Miller, Enrollment Date: 2023-09-05
ID: 6, Name: Fiona Garcia, Enrollment Date: 2024-01-15
ID: 9, Name: Ian Rodriguez, Enrollment Date: 2024-02-01
```

In [125]:
# code
dategte = date(2023,9,2)
datelte = date(2024,2,1)
allStudent = Student.objects.filter(enrollment_date__gte = dategte, enrollment_date__lte = datelte).order_by("enrollment_date")

for i in allStudent:
    print(f"ID: {i.id}, Name: {i.first_name} {i.last_name}, Enrollment Date: {i.enrollment_date}")

ID: 12, Name: Laura Lopez, Enrollment Date: 2023-09-02
ID: 7, Name: George Miller, Enrollment Date: 2023-09-05
ID: 6, Name: Fiona Garcia, Enrollment Date: 2024-01-15
ID: 9, Name: Ian Rodriguez, Enrollment Date: 2024-02-01


19. ค้นหาวิชาที่อยู่ในหมวดหมู่ "Business" หรือ "Technology" และอาจารย์ที่สอนชื่อ "Michael" หรือ "Jessica" แต่ชื่อวิชาห้ามขึ้นต้นด้วย "Introduction" และแสดงรายละเอียดของวิชานั้น ๆ
```
ID: 11, Title: Digital Photography, Teacher: Jessica Miller, Category: Technology, Arts
```

In [156]:
# code
# 0126 for ~
course = Course.objects.get(Q(categories__name = "Business") | Q(categories__name = "Technology"),
                              Q(teacher__first_name = "Michael") | Q(teacher__first_name = "Jessica"),
                              ~Q(title__startswith = "Introduction"))

allcat = course.categories.all()

cat = []
for i in allcat:
    cat.append(i.name)
    
cat = ", ".join(cat)

print(f"ID: {course.id}, Title: {course.title}, Teacher: {course.teacher.first_name} {course.teacher.last_name}, Category: {cat}")

ID: 11, Title: Digital Photography, Teacher: Jessica Miller, Category: Technology, Arts
